In [35]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../helper-files')))

import pandas as pd
from get_task_embeddings import get_task_embeddings
from prompting import prompting
import pandas as pd
import asyncio
import re
import numpy as np
from decimal import Decimal
from llms import gemini
from llms import chatGPT
from sklearn.metrics.pairwise import cosine_similarity
import json
import time

x_chat = chatGPT()
x_gemini = gemini()

prompting_class = prompting()

df = pd.read_csv('../../output/df_with_parsed_description.csv')
df = df.sample(1, random_state=2).reset_index(drop=True)

def read_prompt(file):
    file_path = f"../prompts/{file}.txt"
    with open(file_path, 'r') as file:
        return file.read()
    
example_automation_prompt = read_prompt("example-automation")      

In [36]:

args = [["company","organization name"], ["website","website"], ["description","description_all"]]

async def main(df):
    df = await prompting_class.iterate(df, example_automation_prompt, args, "task_automated")
    df.to_csv('../../output/df_with_task_automated.csv', index=False)
    return df

df = asyncio.run(main(df))

10 0
******************************
Processing 0: Storage Rentals of America


OSError: Cannot save file into a non-existent directory: '../output'

In [286]:
job_cols = [[f"Example_Description{i}",f"Example_Job_Title{i}"] for i in range(1,2)]
for job in job_cols:
    for col in job:
        df = x_chat.run_batch_embeddings(df,col)

NameError: name 'x_chat' is not defined

In [ ]:
task_embeddings, title_embeddings, df_exp = get_task_embeddings().get()

In [14]:
onet_df = pd.read_csv("../../output/onet_df.csv").rename({"organization_name":"organization name"},axis=1)
df = pd.read_csv("../../output/data/retail-data.csv",index_col=0)
df['founded date'] = pd.to_datetime(df['founded date'])
df['year'] = df['founded date'].dt.year
df['tech startup'] =(df['description_all'].str.contains(r'\(AI\)|tech|automate', na=False)) | (df['industries'].str.contains(r'\(AI\)|tech|automate|software', na=False))
df = df[df['tech startup']]
df["AI startup"] = df['industries'].str.contains(r'\(AI\)|tech|automate', na=False)
df = pd.merge(onet_df, df, on="organization name")
len(df)

6192

In [15]:
df = df.groupby("organization name").aggregate({"task_similarity":"max","job_title_similarity":"max","AI startup":"max","year":"max"}).reset_index()


In [16]:
grouped = df.groupby('year').agg({'task_similarity':'mean','job_title_similarity':'mean','organization name':'count','AI startup':'sum'}).reset_index().rename({'organization name':'total number of startups','AI startup':'total number of AI startups'},axis=1)
greater = df[df.job_title_similarity > .75].groupby('year').agg({'organization name':'count'}).reset_index().rename({'organization name':'greater'},axis=1)
grouped = grouped.merge(greater, on="year")
grouped["percent AI startups"] = grouped["total number of AI startups"]/grouped["total number of startups"]
grouped["percent of startups with rating above 70"] = grouped["greater"]/grouped["total number of startups"]
grouped = grouped.drop(columns=["greater"])
grouped

,year,task_similarity,job_title_similarity,total number of startups,total number of AI startups,percent AI startups,percent of startups with rating above 70
0,2010,0.527312,0.667042,107,7,0.065421,0.299065
1,2011,0.530556,0.685998,139,22,0.158273,0.309353
2,2012,0.526792,0.681694,138,18,0.130435,0.326087
3,2013,0.536252,0.692574,151,29,0.192053,0.350993
4,2014,0.536051,0.669516,173,30,0.173410,0.335260
5,2015,0.536172,0.681377,172,31,0.180233,0.372093
6,2016,0.531719,0.685505,178,46,0.258427,0.337079
7,2017,0.534415,0.671187,154,42,0.272727,0.318182
8,2018,0.519990,0.663868,149,36,0.241611,0.315436
9,2019,0.527695,0.670265,148,40,0.270270,0.317568


In [17]:
rating_df = pd.read_csv("../../output/df_with_filter.csv")
df = pd.merge(rating_df, df, on="organization name")

In [21]:
df[(df["automation_rating"]<50) & (df["task_similarity"]>.6)].iloc[2].values

array(['Bods', '2020-01-01', '1-10',
       '3D Technology,E-Commerce,Fashion,Retail,Bods is a fashion technology company that offers a virtual fit solution that allows online shoppers to see how clothes will look and fit in real life by using an AI-generated digital rendering of their actual bodies. Through Bods, users can create their own avatars from two uploaded photographs using computer vision and machine learning, and then virtually fit and try on garments. Bods transforms online shopping into a personalized, immersive, and fun experience.,Marina Del Rey',
       'California', 'United States',
       'Bods is a fashion technology company that provides a virtual fit solution to help online shoppers to determine their fit.',
       'www.bods.me',
       'Bods is a fashion technology company that provides a virtual fit solution to help online shoppers to determine their fit. California',
       "Bods Analysis\n\nRating: 45\n\nReasoning: While Bods automates a part of the shopping e